<a href="https://colab.research.google.com/github/jucearma/ML-Server-Python-Samples/blob/master/Prueba_Tecnica_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import pyspark
import hashlib

In [0]:
from google.colab import drive
drive.mount('/content/drive')

from pyspark.sql import SparkSession, functions as fs


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# 1 - Unificar archivos de ruta /content/drive/My Drive/Julio Cesar Arismendi/Datos/
path = r'/content/drive/My Drive/Julio Cesar Arismendi/Datos'
filesmocks = glob.glob(path + "/*.csv")
txfiles = []

for filemock in filesmocks:
    print(filemock)
    df = pd.read_csv(filemock, index_col=None, header=0)
    txfiles.append(df)

dfTx = pd.concat(txfiles, axis=0, ignore_index=True)
print(dfTx)

export_csv = dfTx.to_csv (r'/content/drive/My Drive/Julio Cesar Arismendi/Datos/all_tx_customer.csv', index = None, header=True)

/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_7.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_4.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_8.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_1.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_6.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_3.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_9.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_5.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_10.csv
/content/drive/My Drive/Julio Cesar Arismendi/Datos/MOCK_DATA_2.csv
        id      nombre  ... numero_identificacion tipo_identificacion
0        1    Yovonnda  ...             475563897                  CC
1        2      Brnaby  ...             323957514                  IT
2        3     Justinn  ...             119614142                  PP
3        4     Urbanus  ...            

In [0]:
print(dfTx)

        id      nombre  ... numero_identificacion tipo_identificacion
0        1    Yovonnda  ...             475563897                  CC
1        2      Brnaby  ...             323957514                  IT
2        3     Justinn  ...             119614142                  PP
3        4     Urbanus  ...             820357068                  IT
4        5      Freddy  ...             149960589                  IT
...    ...         ...  ...                   ...                 ...
9995   996  Clarabelle  ...             199461123                  IT
9996   997        Lela  ...             592418264                  PP
9997   998      Gretta  ...             560913169                  CC
9998   999         Man  ...             750467558                  PP
9999  1000     Charley  ...             665101609                  CC

[10000 rows x 8 columns]


In [0]:
# 2 - Unificacion de Transacciones complementarias
pathComplementary = r'/content/drive/My Drive/Julio Cesar Arismendi/Datos/datos complementos'
filesmocks = glob.glob(pathComplementary + "/*.csv")
txfilesComplementary = []

for filemock in filesmocks:    
    dfc = pd.read_csv(filemock, index_col=None, header=0)
    txfilesComplementary.append(dfc)

dfTxComplementary = pd.concat(txfilesComplementary, axis=0, ignore_index=True)
display(dfTxComplementary)

export_csv = dfTxComplementary.to_csv (r'/content/drive/My Drive/Julio Cesar Arismendi/Datos/all_complementary_tx_customer.csv', index = None, header=True)

,id,nombre,apellido,email,genero,valor_tx,numero_identificacion,tipo_identificacion
0,1,Cal,Hodgins,chodgins0@oakley.com,Female,$121560.79,719027574,PP
1,2,Alis,Collaton,acollaton1@paginegialle.it,Female,$656687.93,313732851,CC
2,3,Maryjane,Aylmer,maylmer2@sourceforge.net,Female,$822627.93,398710389,PP
3,4,Renado,Grisenthwaite,rgrisenthwaite3@sourceforge.net,Male,$647615.81,603854535,IT
4,5,Waite,Stave,wstave4@businessinsider.com,Male,$996902.86,995105757,CC
...,...,...,...,...,...,...,...,...
99995,996,Katie,Tortis,ktortisrn@wikimedia.org,Female,$925975.39,29881361,IT
99996,997,Bernete,Ravenshear,bravenshearro@tuttocitta.it,Female,$792183.77,708892468,CC
99997,998,Barnie,Houlison,bhoulisonrp@ycombinator.com,Male,$853062.13,671112111,CC
99998,999,Clarance,Huey,chueyrq@aol.com,Male,$423416.82,213988673,PP


In [0]:
# Anexar los clientes que esten en el dataframe unificado de complementarios 
# dfTxComplementary al dataframe inicial de transacciones dfTx
# Para lograr esto hare uso de una combinacion cardinal Right Join

addCustomer = pd.merge(dfTx, dfTxComplementary, on='numero_identificacion', how='right')
print(addCustomer)
display(addCustomer)
dfTxComplementary.to_csv (r'/content/drive/My Drive/Julio Cesar Arismendi/Datos/add_customer_not_exists.csv', index = None, header=True)

       id_x  nombre_x  apellido_x  ... genero_y  valor_tx_y tipo_identificacion_y
0       1.0  Yovonnda      Forson  ...   Female  $669582.83                    CC
1       1.0  Yovonnda      Forson  ...   Female  $669582.83                    CC
2       2.0    Brnaby  O'Hartagan  ...     Male  $408267.50                    IT
3       2.0    Brnaby  O'Hartagan  ...     Male  $408267.50                    IT
4       3.0   Justinn   Lindemann  ...   Female  $504026.39                    PP
...     ...       ...         ...  ...      ...         ...                   ...
99995   NaN       NaN         NaN  ...     Male  $130796.87                    PP
99996   NaN       NaN         NaN  ...   Female  $187111.10                    IT
99997   NaN       NaN         NaN  ...   Female  $187111.10                    IT
99998   NaN       NaN         NaN  ...   Female  $221885.99                    IT
99999   NaN       NaN         NaN  ...   Female  $221885.99                    IT

[100000 rows x 

,id_x,nombre_x,apellido_x,email_x,genero_x,valor_tx_x,numero_identificacion,tipo_identificacion_x,id_y,nombre_y,apellido_y,email_y,genero_y,valor_tx_y,tipo_identificacion_y
0,1.0,Yovonnda,Forson,yforson0@cafepress.com,Female,$669582.83,475563897,CC,1,Yovonnda,Forson,yforson0@cafepress.com,Female,$669582.83,CC
1,1.0,Yovonnda,Forson,yforson0@cafepress.com,Female,$669582.83,475563897,CC,1,Yovonnda,Forson,yforson0@cafepress.com,Female,$669582.83,CC
2,2.0,Brnaby,O'Hartagan,bohartagan1@marriott.com,Male,$408267.50,323957514,IT,2,Brnaby,O'Hartagan,bohartagan1@marriott.com,Male,$408267.50,IT
3,2.0,Brnaby,O'Hartagan,bohartagan1@marriott.com,Male,$408267.50,323957514,IT,2,Brnaby,O'Hartagan,bohartagan1@marriott.com,Male,$408267.50,IT
4,3.0,Justinn,Lindemann,jlindemann2@tripod.com,Female,$504026.39,119614142,PP,3,Justinn,Lindemann,jlindemann2@tripod.com,Female,$504026.39,PP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,NaN,NaN,NaN,NaN,NaN,567572674,NaN,998,Thaxter,Yakovlev,tyakovlevrp@hp.com,Male,$130796.87,PP
99996,NaN,NaN,NaN,NaN,NaN,NaN,92505346,NaN,999,Annalise,Reck,areckrq@prweb.com,Female,$187111.10,IT
99997,NaN,NaN,NaN,NaN,NaN,NaN,92505346,NaN,999,Annalise,Reck,areckrq@prweb.com,Female,$187111.10,IT
99998,NaN,NaN,NaN,NaN,NaN,NaN,441729448,NaN,1000,Gladys,Bonsul,gbonsulrr@washingtonpost.com,Female,$221885.99,IT


In [0]:
# Implementacion de clientes no existentes en Spark SQL

session = SparkSession.builder.appName("UkMakerSpaces").master("local[*]").getOrCreate()
alltxcustomer = session.read.option("header", "true") \
        .csv("/content/drive/My Drive/Julio Cesar Arismendi/Datos/all_tx_customer.csv")

alltxcomplementary = session.read.option("header", "true") \
        .csv("/content/drive/My Drive/Julio Cesar Arismendi/Datos/all_complementary_tx_customer.csv")

print("=== Imprimo los primeros registros del Dataframe ===")
alltxcustomer.select("*").show()

addCustomerNotExist = alltxcustomer \
         .join(alltxcomplementary, alltxcustomer["numero_identificacion"].startswith(alltxcomplementary["numero_identificacion"]), "right")

display(addCustomerNotExist)


=== Imprimo los primeros registros del Dataframe ===
+---+--------+----------+--------------------+------+----------+---------------------+-------------------+
| id|  nombre|  apellido|               email|genero|  valor_tx|numero_identificacion|tipo_identificacion|
+---+--------+----------+--------------------+------+----------+---------------------+-------------------+
|  1|Yovonnda|    Forson|yforson0@cafepres...|Female|$669582.83|            475563897|                 CC|
|  2|  Brnaby|O'Hartagan|bohartagan1@marri...|  Male|$408267.50|            323957514|                 IT|
|  3| Justinn| Lindemann|jlindemann2@tripo...|Female|$504026.39|            119614142|                 PP|
|  4| Urbanus|  Ranscomb|  uranscomb3@npr.org|  Male|$145213.17|            820357068|                 IT|
|  5|  Freddy| Garretson|fgarretson4@howst...|  Male|$401426.69|            149960589|                 IT|
|  6|    Nita|Bridywater|nbridywater5@umic...|Female|$530485.15|            181404461|     

DataFrame[id: string, nombre: string, apellido: string, email: string, genero: string, valor_tx: string, numero_identificacion: string, tipo_identificacion: string, id: string, nombre: string, apellido: string, email: string, genero: string, valor_tx: string, numero_identificacion: string, tipo_identificacion: string]

In [0]:
# Concatenar tipo y numero de identificacion en un solo campo
dfTxComplementary['TipoNumIdenticacion'] = dfTxComplementary['tipo_identificacion'] + str(dfTxComplementary['numero_identificacion'])
display(dfTxComplementary)


,id,nombre,apellido,email,genero,valor_tx,numero_identificacion,tipo_identificacion,TipoNumIdenticacion
0,1,Cal,Hodgins,chodgins0@oakley.com,Female,$121560.79,719027574,PP,PP0 719027574\n1 313732851\n2 ...
1,2,Alis,Collaton,acollaton1@paginegialle.it,Female,$656687.93,313732851,CC,CC0 719027574\n1 313732851\n2 ...
2,3,Maryjane,Aylmer,maylmer2@sourceforge.net,Female,$822627.93,398710389,PP,PP0 719027574\n1 313732851\n2 ...
3,4,Renado,Grisenthwaite,rgrisenthwaite3@sourceforge.net,Male,$647615.81,603854535,IT,IT0 719027574\n1 313732851\n2 ...
4,5,Waite,Stave,wstave4@businessinsider.com,Male,$996902.86,995105757,CC,CC0 719027574\n1 313732851\n2 ...
...,...,...,...,...,...,...,...,...,...
99995,996,Katie,Tortis,ktortisrn@wikimedia.org,Female,$925975.39,29881361,IT,IT0 719027574\n1 313732851\n2 ...
99996,997,Bernete,Ravenshear,bravenshearro@tuttocitta.it,Female,$792183.77,708892468,CC,CC0 719027574\n1 313732851\n2 ...
99997,998,Barnie,Houlison,bhoulisonrp@ycombinator.com,Male,$853062.13,671112111,CC,CC0 719027574\n1 313732851\n2 ...
99998,999,Clarance,Huey,chueyrq@aol.com,Male,$423416.82,213988673,PP,PP0 719027574\n1 313732851\n2 ...


In [0]:
# Crear un Hash del campo concatenado en el paso anterior denominado TipoNumIdenticacion
dfTx['hashIdentificacion'] = [hashlib.sha1(str.encode(str(i))).hexdigest() for i in dfTx['numero_identificacion']]
display(dfTx.head())


,id,nombre,apellido,email,genero,valor_tx,numero_identificacion,tipo_identificacion,hashIdentificacion
0,1,Yovonnda,Forson,yforson0@cafepress.com,Female,$669582.83,475563897,CC,d89a57518d4aade19d43b352fd42bfd05cb62669
1,2,Brnaby,O'Hartagan,bohartagan1@marriott.com,Male,$408267.50,323957514,IT,8bdef6944623065e5d64223eb142ef0d550b881b
2,3,Justinn,Lindemann,jlindemann2@tripod.com,Female,$504026.39,119614142,PP,3ec867f69c556cbcbd4b7c092132a69da2f9951c
3,4,Urbanus,Ranscomb,uranscomb3@npr.org,Male,$145213.17,820357068,IT,20dd57f1138ec25c0e76bcbeaf5757e2d5deffb3
4,5,Freddy,Garretson,fgarretson4@howstuffworks.com,Male,$401426.69,149960589,IT,870c19152b90c71e381fc2944bf84b5d2fb8fc1c
